## Table of contents:

1. [Importing libraries](#Libraries)
2. [Loading data](#Data)
3. [Statistical summaries](#Statistics)
4. [Missing values and duplicates](#Missing)
5. [Outliers](#Outliers)
6. [Feature engineering](#Engineering)
7. [Date features EDA](#Dates)
8. [Correlations - EDA](#Correlations)
9. [Preprocess test dataset](#Preprocess)
10. [Modelling](#Modelling)
11. [Making predictions of the test set and creating a submission file](#Predictions)
12. [TO DOs](#Tips)

<a name="Libraries"></a>
## 1. Importing libraries


In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, ConfusionMatrixDisplay
pd.options.display.float_format = '{:.5f}'.format

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
np.random.seed(42)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

<a name="Data"></a>
## 2. Loading data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_path='/content/drive/MyDrive/#UmojaHackAfrica22/train.csv'
test_path='/content/drive/MyDrive/#UmojaHackAfrica22/test.csv'
samplesubmission_path='/content/drive/MyDrive/#UmojaHackAfrica22/SampleSubmission.csv'

In [ ]:
# Load files
train = pd.read_csv(train_path, parse_dates = ['Datetime'])
test = pd.read_csv(test_path, parse_dates = ['Datetime'])
samplesubmission = pd.read_csv(samplesubmission_path)

# Preview train dataset
train.head()

FileNotFoundError: ignored

In [ ]:
# Preview test dataset
test.head()

In [ ]:
# Preview sample submission file
samplesubmission.head()

In [ ]:
train["difference"] = abs((train['Sensor1_PM2.5']-train['Sensor2_PM2.5']))
train.head()
train[train["Offset_fault"]==1]

In [ ]:
test["difference"] = abs((test['Sensor1_PM2.5']-test['Sensor2_PM2.5']))
test.head()


In [ ]:
# Check size and shape of datasets
train.shape, test.shape, samplesubmission.shape

<a name="Statistics"></a>
## 3. Statistical summaries

In [ ]:
# Train statistical summary
train.describe(include = 'all')

From the above statistical summary, we can deduce some of the following insights:
 - The train data provided ranges from *2021-10-15 16:00:34* to *2022-01-21 07:34:57*
 - There is a high correlation between Sensor1_PM2.5	and Sensor2_PM2.5
 - Minimum recorded temperature is *16.70000* and a maximum *34.90000*

In [ ]:
# Target variable distribution
sns.set_style('darkgrid')
plt.figure(figsize=(8, 5))
sns.countplot(train["Offset_fault"])
plt.title('Target variable distribution')
plt.show()


The target variable is not well balanced.
Some of the techniques to handle imbalanceness include:
- Smote
- Oversampling
- Undersampling ...

<a name="Missing"></a>
## 4. Missing values and duplicates

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(train.isna())

In [ ]:
# Check for missing values
train.isnull().sum().any(), test.isnull().sum().any() 

In [ ]:
# Plot missing values in train set
ax = train.isna().sum().sort_values().plot(kind = 'barh', figsize = (9, 10))
plt.title('Percentage of Missing Values Per Column in Train Set', fontdict={'size':15})
for p in ax.patches:
    percentage ='{:,.0f}%'.format((p.get_width()/train.shape[0])*100)
    width, height =p.get_width(),p.get_height()
    x=p.get_x()+width+0.02
    y=p.get_y()+height/2
    ax.annotate(percentage,(x,y))

In [ ]:
print(train.isnull().sum())

In [ ]:
test.isna().sum()

In [ ]:
train['hour']=train.Datetime.dt.hour

for column in ['Temperature','Relative_Humidity']:
    median = train.groupby('hour')[column].transform('median')

    train[column].fillna(median, inplace=True)
print(train.isnull().sum())

In [ ]:
train['hour']=train.Datetime.dt.hour

for column in ['Temperature','Relative_Humidity']:
    median = train.groupby('hour')[column].transform('median')

    test[column].fillna(median, inplace=True)


In [ ]:

print(test.isnull().sum())

In [ ]:
train = train.dropna()
print(train.isnull().sum())

Suggestions on how to handle missing values:
 - Fill in missing values with mode, mean, median..
 - Drop Missing datapoints with missing values
 - Fill in with a large number e.g -999999

In [ ]:
# Check for duplicates
train.duplicated().any(), test.duplicated().any()

No duplictes 

In [ ]:
train.shape

<a name="Outliers"></a>
## 5. Outliers

Outliers are those data points which differs significantly from other observations present in given dataset.

Suggestions on how to handle outliers:
 - Transforming the outliers by scaling - log transformation, box-cox transformation ...
 - Dropping outliers
 - Imputation by replacing outliers with mean, median ...


In [ ]:
print(train.quantile(0.05))
print(train.quantile(0.95))
num_col = train.select_dtypes(include=['int64','float64']).columns.tolist()

# or you can create a custom list of numerical columns

train[num_col]=train[num_col].apply(lambda x: x.clip(*x.quantile([0.05, 0.95])))

In [ ]:
num_col=test.select_dtypes(include=['int64','float64']).columns.tolist()

In [ ]:
test[num_col]=test[num_col].clip(train[num_col].quantile(0.05), train[num_col].quantile(0.95), axis=1)

In [ ]:
# print(test.quantile(0.05))
# print(test.quantile(0.95))
# num_col = test.select_dtypes(include=['int64','float64']).columns.tolist()

# # or you can create a custom list of numerical columns

# test[num_col] = test[num_col].apply(lambda x: x.clip(*x.quantile([0.05, 0.95])))

In [ ]:
train.drop('hour',inplace=True,axis=1)

In [ ]:
# Plotting boxplots for each of the numerical columns
sns.set_style('darkgrid')
fig, axes = plt.subplots(nrows = 2, ncols = 2, figsize = (15, 10))
fig.suptitle('Box plots showing outliers', y= 0.93, fontsize = 15)

for ax, data, name in zip(axes.flatten(), train, ['Sensor1_PM2.5',	'Sensor2_PM2.5',	'Temperature',	'Relative_Humidity']):
  sns.boxplot(train[name], ax = ax)

<a name="Engineering"></a>
## 6. Feature engineering

In [ ]:
# Extract day, month year and hour from the Datetime column
# day

# month
train['Datetime_month'] = train.Datetime.dt.month
 
# hour
train['Datetime_hour'] = np.sin(train.Datetime.dt.hour*(np.pi/23))


# Preview engineered date features


In [ ]:
# Extract day, month year and hour from the Datetime column
# day

# month
test['Datetime_month'] = test.Datetime.dt.month
 
# hour
test['Datetime_hour'] = np.sin(test.Datetime.dt.hour*(np.pi/23))


# Preview engineered date features
test[['Datetime', 'Datetime_month', 'Datetime_hour']].head()

<a name="Dates"></a>
## 7. Date features EDA

In [ ]:
# Datetime month countplot
plt.figure(figsize = (14, 7))
sns.countplot(x = 'Datetime_month', data = train)
plt.title('Datetime month count plot')
plt.show()

- The only months available in the train set include *Jan, Oct, Nov and Dec*
- March has the least number of observations in the dataset while December has the highest number of observations

In [ ]:
# Box plots for Sensor1_PM2.5	and Sensor2_PM2.5 vs Offset faults
sns.set_style('darkgrid')
fig, axes = plt.subplots(nrows = 1, ncols = 2, figsize = (17, 7))
fig.suptitle('Box plot for Sensor1_PM2.5	and Sensor2_PM2.5 vs Offset faults', y= 0.95, fontsize = 15)

for ax, data, name in zip(axes.flatten(), train, ['Sensor1_PM2.5', 'Sensor2_PM2.5']):
  sns.boxplot(train.Offset_fault, train[name], ax= ax)

 - Sensor 1 and sensor 2 are highly correlated with majority of the non faulty observations having the highest number of outliers

<a name="Correlations"></a>
## 8. Correlations - EDA

In [ ]:
# Type of correlations 
plt.figure(figsize = (20, 12))
num_cols = ['Sensor1_PM2.5',	'Sensor2_PM2.5',	'Temperature',	'Relative_Humidity','Datetime_hour']
sns.pairplot(train[num_cols], kind="scatter", plot_kws=dict(s=80, edgecolor="white", linewidth=2.5))
plt.show()

 - There is a positive correlation between Sensor1_PM2.5	Sensor2_PM2.5
 - There is a negative correlation between temperature and humidity
 - There seems to be no correlation between sensor PM2.5 and temperature/humidity

In [ ]:
# Quantify correlations
corr = train.corr()
plt.figure(figsize = (13, 8))
sns.heatmap(corr, cmap='RdYlGn', annot = True, center = 0)
plt.title('Correlogram', fontsize = 15, color = 'darkgreen')
plt.show()

- There is a 0.96 positive correlation between sensor1 PM2.5 and sensor2 PM2.5
- There is a -0.99 negative correlation between temperature and humidity

In [ ]:
train.info()


In [ ]:
!pip install imbalanced-learn

In [ ]:
# check version number
import imblearn
print(imblearn.__version__)

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
train['avg_sensors']=(train['Sensor1_PM2.5']+train['Sensor2_PM2.5'])/2
test['avg_sensors']=(test['Sensor1_PM2.5']+test['Sensor2_PM2.5'])/2

In [ ]:
X=train.drop(['Offset_fault','ID','Datetime','Relative_Humidity'],axis=1)
y=train['Offset_fault']
X_train,X_test, y_train,y_test=train_test_split(X,y)

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

sss=StratifiedShuffleSplit(n_splits=1,test_size=0.2,random_state=42)
for train,test in sss.split(X,y):
  X_train,X_test=X.iloc[train],X.iloc[test]
  y_train,y_test=y.iloc[train],y.iloc[test]

In [ ]:
min_max_scaler = MinMaxScaler()
X_train_minmax = min_max_scaler.fit_transform(X_train)

In [ ]:
X_test_minmax = min_max_scaler.fit_transform(X_test)

In [ ]:
oversample = SMOTE()
X_train_minmax, y_train = oversample.fit_resample(X_train_minmax, y_train)

In [ ]:
oversample = SMOTE()
X_train, y_train = oversample.fit_resample(X_train, y_train)

In [ ]:
# (pour entrainer sur toute la BDD Train) X_minmax = min_max_scaler.fit_transform(X)

In [ ]:
# Target variable distribution
sns.set_style('darkgrid')
plt.figure(figsize=(8, 5))
sns.countplot(y_train)
plt.title('Target variable distribution')
plt.show()

In [ ]:
## (pour entrainer sur toute la BDD Train)
#oversample = SMOTE()
#X_minmax, y = oversample.fit_resample(X_minmax, y)

#sns.set_style('darkgrid')
#plt.figure(figsize=(8, 5))
#sns.countplot(y)
#plt.title('Target variable distribution')
#plt.show()

<a name="Modelling"></a>
## 10.  Modelling

In [ ]:
# from sklearn.linear_model import LogisticRegression
# logreg = LogisticRegression()
# logreg.fit(X_train_minmax, y_train)


In [ ]:
# neigh = KNeighborsClassifier(11)
# neigh.fit(X_train_minmax, y_train)


In [ ]:


# # Instantiating the model
# clf = RandomForestClassifier(max_depth = 3, random_state = 0, n_estimators=200)
# clf.fit(X_train_minmax, y_train)

##Fine Tuning GradientBoostingClassifier

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier()
gbc.fit(X_train, y_train)


In [ ]:
# Making predictions
y_pred = gbc.predict(X_test)

# Measuring the accuracy of the model
print(f'Accuracy score: {accuracy_score(y_test, y_pred)}')
print('\n')
print(f'{classification_report(y_test, y_pred)}')

In [ ]:
# Confusion matrix
cm = confusion_matrix(y_test, y_pred, labels=gbc.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=gbc.classes_)
fig, ax = plt.subplots(figsize=(15,7))
disp.plot(ax=ax)
plt.show()

In [ ]:
# Feature importance
impo_df = pd.DataFrame({'feature': X.columns, 'importance': neigh.feature_importances_}).set_index('feature').sort_values(by = 'importance', ascending = False)
impo_df = impo_df[:12].sort_values(by = 'importance', ascending = True)
impo_df.plot(kind = 'barh', figsize = (10, 10), color = 'purple')
plt.legend(loc = 'center right')
plt.title('Bar chart showing feature importance', color = 'indigo', fontsize = 14)
plt.xlabel('Features', fontsize = 12, color = 'indigo')
plt.show()

<a name="Predictions"></a>
## 11. Making predictions of the test set and creating a submission file

In [ ]:
test_df = test.drop(['ID','Datetime','Relative_Humidity'],axis=1)
# test_minmax = min_max_scaler.fit_transform(test_df)
# test_minmax

In [ ]:
# Make prediction on the test set

predictions = gbc.predict(test_df)

# # Create a submission file
sub_file = samplesubmission.copy()
sub_file.Offset_fault = predictions

# Check the distribution of your predictions
plt.figure(figsize=(8, 5))
sns.countplot(sub_file.Offset_fault)
plt.title('Predictions Data Distribution');

In [ ]:
sub_file

 - Majority of the model predictions are 0

In [ ]:
# Create file
sub_file.to_csv('sub_gboost7.csv', index = False)